# Working with Enriched Events data using Spark SQL

In this Lab we'll learn how to work with [Enriched Event Data](https://docs.developers.optimizely.com/web/docs/enriched-events-export) using [PySpark](http://spark.apache.org/docs/latest/api/python/index.html) and [Spark SQL](http://spark.apache.org/sql/).

This lab contains a set of simple, useful queries for working with this dataset.  These queries can help you answer questions like
- How many visitors were tested in my experiment?
- How many "unique conversions" of an event were attributed to this variation?
- What is the total revenue attributed to this variation?

This Lab covers some of the basics of working with event-level experiment data. There are many more useful questions may want to answer with experiment data.  Future tutorials will go deeper on the topic.

This guide borrows some initialization code from the [Spark SQL getting started guide](https://spark.apache.org/docs/latest/sql-getting-started.html).

## Creating a Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

## Loading enriched event data

We'll start by loading decision and conversion data from `/data`.

In [2]:
spark.read.parquet("data/decisions").createOrReplaceTempView("decisions")
spark.read.parquet("data/events").createOrReplaceTempView("events")

## Querying our data

Now that we've loaded our data, we can query it using the `sql()` function.  Here's an example on our `decisions` view:

In [3]:
spark.sql("""
    SELECT
        *
    FROM decisions
    LIMIT 5
""")

uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,campaign_id,experiment_id,variation_id,attributes,user_ip,user_agent,referer,is_holdback,revision,client_engine,client_version
F4F1EF48-6BC2-415...,2020-05-25 15:27:...,2020-05-25 15:29:...,visitor_159044565...,-1235693267,596780373,18149940006,18156943409,18174970251,"[[100,, browserId...",75.111.77.0,Mozilla/5.0 (Maci...,https://app.optim...,false,null,ricky/fakedata.pwned,1.0.0
F76EC706-CEEB-4DD...,2020-05-25 15:27:...,2020-05-25 15:28:...,visitor_159044565...,1572977898,596780373,18149940006,18156943409,18112613000,"[[100,, browserId...",75.111.77.0,Mozilla/5.0 (Maci...,https://app.optim...,false,null,ricky/fakedata.pwned,1.0.0
B126B61B-E4CB-481...,2020-05-25 15:27:...,2020-05-25 15:29:...,visitor_159044565...,-1253110529,596780373,18149940006,18156943409,18112613000,"[[100,, browserId...",75.111.77.0,Mozilla/5.0 (Maci...,https://app.optim...,false,null,ricky/fakedata.pwned,1.0.0
74182819-6E3E-4CA...,2020-05-25 15:27:...,2020-05-25 15:28:...,visitor_159044565...,1349491186,596780373,18149940006,18156943409,18174970251,"[[100,, browserId...",75.111.77.0,Mozilla/5.0 (Maci...,https://app.optim...,false,null,ricky/fakedata.pwned,1.0.0
843CA8A3-DBAB-497...,2020-05-25 15:27:...,2020-05-25 15:29:...,visitor_159044565...,1203115117,596780373,18149940006,18156943409,18174970251,"[[100,, browserId...",75.111.77.0,Mozilla/5.0 (Maci...,https://app.optim...,false,null,ricky/fakedata.pwned,1.0.0


Here's an example on our `events` view:

In [4]:
spark.sql("""
    SELECT
        *
    FROM events
    LIMIT 5
""")

uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,experiments,entity_id,attributes,user_ip,user_agent,referer,event_type,event_name,revenue,value,quantity,tags,revision,client_engine,client_version
235ABEC8-C9A1-448...,2020-05-24 17:34:...,2020-05-24 17:41:...,visitor_159036686...,-1274245065,596780373,"[[18128690585, 18...",15776040040,"[[100,, browserId...",174.222.139.0,Mozilla/5.0 (Maci...,https://app.optim...,null,add_to_cart,1000,1000.00001,null,[],null,ricky/fakedata.pwned,1.0.0
62EDB4C0-C72F-490...,2020-05-24 17:34:...,2020-05-24 17:37:...,visitor_159036686...,-1274245065,596780373,"[[18128690585, 18...",15776040040,"[[100,, browserId...",174.222.139.0,Mozilla/5.0 (Maci...,https://app.optim...,null,add_to_cart,0,0.0,null,[],null,ricky/fakedata.pwned,1.0.0
6CC5EAE5-8C51-4D3...,2020-05-24 17:34:...,2020-05-24 17:42:...,visitor_159036686...,-1274245065,596780373,"[[18128690585, 18...",15776040040,"[[100,, browserId...",174.222.139.0,Mozilla/5.0 (Maci...,https://app.optim...,null,add_to_cart,1000,1000.00001,null,[],null,ricky/fakedata.pwned,1.0.0
16512193-2F2A-4AA...,2020-05-24 17:34:...,2020-05-24 17:38:...,visitor_159036686...,-1274245065,596780373,"[[18128690585, 18...",15776040040,"[[100,, browserId...",174.222.139.0,Mozilla/5.0 (Maci...,https://app.optim...,null,add_to_cart,0,0.0,null,[],null,ricky/fakedata.pwned,1.0.0
C526A693-C963-46C...,2020-05-24 17:34:...,2020-05-24 17:39:...,visitor_159036687...,2119343889,596780373,"[[18128690585, 18...",15776040040,"[[100,, browserId...",174.222.139.0,Mozilla/5.0 (Maci...,https://app.optim...,null,add_to_cart,0,0.0,null,[],null,ricky/fakedata.pwned,1.0.0


## Useful queries

Next we'll cover some simple, useful queries for working with Optimizely's Enriched Event data. 

These queries are parameterized with the following values:

In [5]:
start = "2020-05-24 00:00:00"
end = "2020-05-26 23:59:59"

### Counting the unique visitors in an Optimizely Web experiment 

[Optimizely Web]: https://www.optimizely.com/platform/experimentation/
[Optimizely Full Stack]: https://docs.developers.optimizely.com/full-stack/docs

[Optimizely Web] and [Optimizely Full Stack] experiment results pages count unique visitors in slightly different ways.  

Given a particular analysis time window (between `start` and `end`) [Optimizely Web] attributes all visitors who were exposed to a variation at any time between when the experiment started and `end` and sent _any_ event (decision or conversion) to Optimizely between `start` and `end`.

The following query captures that attribution logic:

In [6]:
# Count the unique visitors from all events (Optimizely Web)

spark.sql(f"""
    SELECT 
        experiment_id,
        variation_id,
        COUNT (distinct visitor_id) as `Unique visitors (Optimizely Web)`
    FROM (
        SELECT
            exp.experiment_id as experiment_id,
            exp.variation_id as variation_id,
            visitor_id
        FROM events
        LATERAL VIEW explode(experiments) t AS exp
        WHERE timestamp between '{start}' AND '{end}'
        UNION
        SELECT
            experiment_id,
            variation_id,
            visitor_id
        FROM decisions
        WHERE
            timestamp between '{start}' AND '{end}'
            AND is_holdback = false
        )
    GROUP BY
        experiment_id,
        variation_id
    ORDER BY
        experiment_id ASC,
        variation_id ASC
""")

experiment_id,variation_id,Unique visitors (Optimizely Web)
18142600572,18130191769,972
18142600572,18159011346,963
18156943409,18112613000,4487
18156943409,18174970251,4514


**A note on `timestamp` vs `process_timestamp`:** If you're working on re-computing the numbers you see on your [experiment results page](https://help.optimizely.com/Analyze_Results/The_Experiment_Results_page_for_Optimizely_X), it's important to understand the difference between the `timestamp` and `process_timestamp` fields in your Enriched Events data.

- `timestamp` contains the time set by the _client_, e.g. the Optimizely Full Stack SDK
- `process_timestamp` contains the approximate time that the event payload was received by Optimizely

The difference is important because Enriched Event data is partitioned by `process_timestamp`, but Optimizely results are computed using `timestamp`.  This allows clients to send events retroactively, but also means that depending on your implementation you may need to load a wider range of data in order to ensure that you've captured all of the events with a `timestamp` in your desired analysis range.

### Counting the unique visitors in an Optimizely Full Stack experiment 

[Optimizely Web]: https://www.optimizely.com/platform/experimentation/
[Optimizely Full Stack]: https://docs.developers.optimizely.com/full-stack/docs

The [Full Stack][Optimizely Full Stack] attribution model is a little simpler:

Given a particular analysis time window (between `start` and `end`) [Full Stack][Optimizely Full Stack] attributes all visitors who were exposed to a variation at any time between `start` and `end`.  We measure this by counting the unique `visitor_id`s in the decisions dataset for that experiment:

In [7]:
# Count the unique visitors from decisions (Optimizely Full Stack)

spark.sql(f"""
    SELECT
        experiment_id,
        variation_id,
        COUNT(distinct visitor_id) as `Unique visitors (Full Stack)`
    FROM decisions
    WHERE
        timestamp between '{start}' AND '{end}'
        AND is_holdback = false
    GROUP BY
        experiment_id,
        variation_id
""")

experiment_id,variation_id,Unique visitors (Full Stack)
18156943409,18174970251,4514
18156943409,18112613000,4487


### Counting conversions in an Optimizely Web experiment

[Optimizely Web]: https://www.optimizely.com/platform/experimentation/
[Optimizely Full Stack]: https://docs.developers.optimizely.com/full-stack/docs

When it comes to counting conversions, [Optimizely Full Stack] and [Optimizely Web] do things a little differently.

Given a particular analysis time window (between `start` and `end`) [Optimizely Web] will attribute an event to a particular variation if the visitor who triggered that event was exposed to the variation at any time prior to that event, _even if it was before the beginning of the analysis time window._

Optimizely event data is enriched with a an attribution column, `experiments`, that lists all of the experiments and variations to which an event has been attributed. Since Optimizely Web does not require that a corresponding decision take place during the analysis window, we can use a simple query to count the number of attributed conversions during our analysis window.

In [8]:
# Count the unique conversions of a particular event attributed to an experiment

spark.sql(f"""
    SELECT 
        exp.experiment_id as experiment_id,
        exp.variation_id as variation_id,
        event_name,
        COUNT(1) as `Conversion count (Optimizely Web)`
    FROM events
    LATERAL VIEW explode(experiments) t AS exp
    WHERE 
        timestamp between '{start}' AND '{end}'
    GROUP BY
        experiment_id, variation_id, event_name
    ORDER BY
        experiment_id ASC,
        variation_id ASC,
        event_name ASC
""")

experiment_id,variation_id,event_name,Conversion count (Optimizely Web)
18142600572,18130191769,add_to_cart,2958
18142600572,18159011346,add_to_cart,2919
18156943409,18112613000,add_to_cart,2577
18156943409,18174970251,add_to_cart,2655


### Counting conversions in an Optimizely Full Stack experiment

[Optimizely Web]: https://www.optimizely.com/platform/experimentation/
[Optimizely Full Stack]: https://docs.developers.optimizely.com/full-stack/docs

Given a particular analysis time window (between `start` and `end`) [Optimizely Full Stack] will attribute an event to a particular variation if the visitor who triggered that event was exposed to the variation prior to that event and _during the analysis window._

Since Optimizely Full Stack requires that a corresponding decision take place during the analysis window, the query required to attribute events to experiments and variation is more complex.

In [9]:
spark.sql(f"""
    SELECT 
        experiment_id,
        variation_id,
        event_name,
        COUNT (1)
    FROM (
         SELECT 
             d.experiment_id,
             d.variation_id,
             e.event_name,
             e.visitor_id
         FROM events e
         INNER JOIN 
         (
            SELECT 
                experiment_id,
                variation_id,
                visitor_id,
                MIN(timestamp) as decision_timestamp
            FROM decisions
            WHERE 
                timestamp between '{start}' AND '{end}'
                AND is_holdback = false
            GROUP BY
                experiment_id,
                variation_id,
                visitor_id
         ) d 
         ON e.visitor_id = d.visitor_id
         WHERE
             e.timestamp  between '{start}' AND '{end}'
             AND e.timestamp >= d.decision_timestamp
    )
    GROUP BY
         experiment_id,
         variation_id,
         event_name
""")

experiment_id,variation_id,event_name,count(1)
18156943409,18174970251,add_to_cart,2655
18156943409,18112613000,add_to_cart,2577
